In [1]:
import giraffe
import numpy as np
import pandas as pd

In [2]:
ppi = pd.read_csv("luad_data_processing/ppi_matrix.csv", index_col = 0)
prior = pd.read_csv("luad_data_processing/prior_matrix.csv", index_col = 0)

In [3]:
expr = pd.read_csv("luad_data_processing/raw/expression.csv", index_col = 0)
genes = [gene[0:15] for gene in expr.index]
expr.index = genes
expr = expr.sort_index()
expr_lcss = pd.read_csv("luad_data_processing/expr_lcss.csv", index_col = 0)
expr_lcss = expr_lcss[~expr_lcss.index.duplicated(keep='first')].dropna()
expr_gse = pd.read_csv("luad_data_processing/expression_GEO.csv", index_col = 0)
expr_gse = expr_gse[~expr_gse.index.duplicated(keep='first')].dropna()

## GIRAFFE

In [6]:
R_tcga = giraffe.Giraffe(expr.to_numpy(), prior.to_numpy(), ppi.to_numpy()).get_regulation()

In [4]:
R_lcss = giraffe.Giraffe(
    expr_lcss.iloc[expr_lcss.index.isin(expr.index),],
    prior.iloc[expr.index.isin(expr_lcss.index),].to_numpy(),
    ppi.to_numpy()
).get_regulation()

In [9]:
R_gse = giraffe.Giraffe(
    expr_gse.iloc[expr_gse.index.isin(expr.index),],
    prior.iloc[expr.index.isin(expr_gse.index),].to_numpy(),
    ppi.to_numpy()
).get_regulation()

In [13]:
R_tcga = pd.DataFrame(R_tcga, index = expr.index)
R_lccs = pd.DataFrame(R_lcss, index = expr_lcss.iloc[expr_lcss.index.isin(expr.index),].index)
R_gse = pd.DataFrame(R_gse, index = expr_gse.iloc[expr_gse.index.isin(expr.index),].index)

In [17]:
first = np.corrcoef(
    R_tcga.iloc[R_tcga.index.isin(R_lccs.index),].to_numpy().flatten(),
    R_lccs.iloc[R_lccs.index.isin(R_tcga.index),].to_numpy().flatten()
)[0, 1]
second = np.corrcoef(
    R_tcga.iloc[R_tcga.index.isin(R_gse.index),].to_numpy().flatten(),
    R_gse.iloc[R_gse.index.isin(R_tcga.index),].to_numpy().flatten()
)[0, 1]
third = np.corrcoef(
    R_gse.iloc[R_gse.index.isin(R_lccs.index),].to_numpy().flatten(),
    R_lccs.iloc[R_lccs.index.isin(R_gse.index),].to_numpy().flatten()
)[0, 1]
res_giraffe = [first, second, third]

In [18]:
res_giraffe

[0.9591557577118879, 0.8673521433627063, 0.9084230763358899]

## OTTER

In [4]:
from netZooPy.otter.otter import otter

In [17]:
R_tcga = otter(prior.to_numpy().T, ppi.to_numpy(), np.corrcoef(expr.to_numpy()), Iter=200).T

In [18]:
R_lcss = otter(
    prior.iloc[expr.index.isin(expr_lcss.index),].to_numpy().T,
    ppi.to_numpy(),
    np.corrcoef(expr_lcss.iloc[expr_lcss.index.isin(expr.index),].to_numpy()), 
    Iter=200
).T

In [19]:
R_gse = otter(
    prior.iloc[expr.index.isin(expr_gse.index),].to_numpy().T,
    ppi.to_numpy(),
    np.corrcoef(expr_gse.iloc[expr_gse.index.isin(expr.index),].to_numpy()),
    Iter=200
).T

In [20]:
R_tcga = pd.DataFrame(R_tcga, index = expr.index)
R_lccs = pd.DataFrame(R_lcss, index = expr_lcss.iloc[expr_lcss.index.isin(expr.index),].index)
R_gse = pd.DataFrame(R_gse, index = expr_gse.iloc[expr_gse.index.isin(expr.index),].index)

In [21]:
first = np.corrcoef(
    R_tcga.iloc[R_tcga.index.isin(R_lccs.index),].to_numpy().flatten(),
    R_lccs.iloc[R_lccs.index.isin(R_tcga.index),].to_numpy().flatten()
)[0, 1]
second = np.corrcoef(
    R_tcga.iloc[R_tcga.index.isin(R_gse.index),].to_numpy().flatten(),
    R_gse.iloc[R_gse.index.isin(R_tcga.index),].to_numpy().flatten()
)[0, 1]
third = np.corrcoef(
    R_gse.iloc[R_gse.index.isin(R_lccs.index),].to_numpy().flatten(),
    R_lccs.iloc[R_lccs.index.isin(R_gse.index),].to_numpy().flatten()
)[0, 1]
res_otter = [first, second, third]

In [22]:
res_otter

[0.986878596479477, 0.9863352531378926, 0.99957752059451]

## Pearson correlation

In [21]:
import notebooks

In [23]:
R_panda = notebooks.compute_panda_locally(expr.to_numpy(), prior.to_numpy(), ppi.to_numpy())

KeyboardInterrupt: 